# LLM examples

## Learning goals: 

- Understand the concept and mechanisms of "tools" in the context of LLMs by examples. 

In this notebook, we will manually create a mechanism to call an external "tool" to complement the LLM's output. This is an exercise to understand the prompting strategy. 

(In practice, most LLMs now have systematic ways to specify new tools and each provides a specific syntax to specify tools. Also note that working with tools, and some of the related prompting strategies, are very recent and there is still active research in this). 

## Dependencies

We will use the packages `cohere` (see notebook 1) and `openfoodfacts`. Make sure these are installed. 

## Environment preparation

In [1]:
import cohere
import openfoodfacts

In [2]:
COHERE_API_KEY = # YOUR API KEY
cohere_client = cohere.Client(COHERE_API_KEY)
api = openfoodfacts.API(user_agent="MyAwesomeApp/1.0")


## Working with tools

Imagine we ask Cohere about the calories of some food item. While it will still provide an answer, this answer will be based on the word statistics learned by the model. 

In [3]:
prompt = input("Enter some text")

response = cohere_client.chat(
    message=prompt,
)

language = response.text
print(language)

Mikado, the chocolate-covered biscuit sticks, typically contain around 490 calories per 100 grams. However, the exact calorie count can vary slightly depending on the specific variety and serving size. It's always a good idea to check the nutrition label on the package for the most accurate information.


Let's say, in order to have a more accurate answer, we want to connect this LLM with the OpenFoodFacts database. 

First, we create a function that queries the database given the name of the product and returns the calories: 

In [4]:
def get_calories(name):
    """
    Get the calories of a product by its name.
    """
    result = api.product.text_search(name)

    # This just takes the first item of the search. 
    # In a real application, you would need to do more error checking.
    first_match = result['products'][0]
    id = first_match['_id']
    result = api.product.get(id, fields=["code", "product_name", "energy-kcal_100g"])
    return result['energy-kcal_100g']


Now we can use the following strategy for the LLM to call this function if needed. 

- We do a first call to the LLM to check if the prompt asks about calories. If not, respond normally. 
- If the prompt asks about calories, respond with some internal "code" so that we can intercept it and do further processing. 
- If we detect the code, then we call the above function. 
- Finally, we create a new prompt  asking the LLM to provide a final response based on the original request and the information returned by the function. 

In [5]:
message = input("Enter some text")
prompt = """
This is a normal chatbot. Except if it the user asks for the calories of a food item,
return the message CALORIES_REQUEST followed by the product name,
For example, if the user asks for the calories of an apple, return CALORIES_REQUEST apple.
Now comes the text that the user entered: 
"""

response = cohere_client.chat(
    message=prompt+ message,
)

output = response.text

if output.startswith("CALORIES_REQUEST"):
    product_name = output.split(" ")[1]
    calories = get_calories(product_name)
    print("<INTERMEDIATE RESULT> Calories=", calories)
    final_response = cohere_client.chat(
        message=f"""
            Now complete the request of the user, knowing 
            the calories of the product on the OpenFoodFacts database is {calories}.
            Mention this source in the response.
        """,
        chat_history=[
            {"role": "user", "message": message}
        ])
    print("Final response: ", final_response.text)
else:
    print("Final response: ", output)



<INTERMEDIATE RESULT> Calories= 484
Final response:  Each serving of Mikado, which is typically a pack of 13 mini biscuit sticks, contains approximately 484 calories. This information is provided by the OpenFoodFacts database. It's important to note that the calorie content may vary slightly depending on the specific Mikado product and serving size.


## Web search

The mechanism to call tools is embedded into most LLM packages. (Although The use of tools is recent and there is still research going on about specific prompting strategies). 

One commonly used tool is the web search. You can ask the LLM to perform a web search to get additional information before compiling the final answer. 

In cohere, that is quite straightforward using adding `web-search` to the `connectors` list, as exemplified now. 


In [18]:
message = "Is Flick the new coach of FC Barcelona for the next season?"
response = cohere_client.chat(
    message=message,
    connectors = [{"id": "web-search"}]
)
response.text

'Hansi Flick has been announced as the new head coach of FC Barcelona. He replaces club legend Xavi Hernandez, who was dismissed before the weekend. Flick has signed a contract with the club until June 30, 2026.'

(Try without the web search and you might get outdated info)

The response object can be inspected to get information of the sources used to compile the response: 

In [16]:
print(response.json(indent=2))

{
  "text": "According to sources, Xavi Hernandez will step down as the coach of FC Barcelona on June 30, 2024. Hansi Flick is one of the potential candidates to replace Xavi, but no official announcement has been made yet. Therefore, it cannot be said with certainty that Flick will be the new coach of FC Barcelona for the next season.",
  "generation_id": "1ad8d9e3-1f18-4517-a32a-156a5b88ce98",
  "citations": [
    {
      "start": 22,
      "end": 36,
      "text": "Xavi Hernandez",
      "document_ids": [
        "web-search_1",
        "web-search_2",
        "web-search_4",
        "web-search_5"
      ]
    },
    {
      "start": 42,
      "end": 51,
      "text": "step down",
      "document_ids": [
        "web-search_2",
        "web-search_5"
      ]
    },
    {
      "start": 84,
      "end": 98,
      "text": "June 30, 2024.",
      "document_ids": [
        "web-search_5"
      ]
    },
    {
      "start": 99,
      "end": 110,
      "text": "Hansi Flick",
      "docume